# Яндекс Музыка

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

В рамках проекта Яндекс Музыки произведено сравнение поведения пользователей двух столиц.

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные о поведении пользователей получены из файла `yandex_music_project.csv`, содержащего информацию о прослушиваниях треков в сервисе Яндекс.Музыка. О качестве данных ничего не известно, поэтому перед проверкой гипотез понадобится обзор данных. 

В рамках проекта данные необходимо проверить на ошибки и оценить их влияние на исследование, а на этапе предобработки — провести поиск возможности исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.


## Обзор данных Яндекс Музыки

In [1]:
import pandas as pd # импорт библиотеки pandas

In [2]:
df = pd.read_csv('datasets/yandex_music_project.csv')  # чтение файла с данными и сохранение в датафрейм df

In [3]:
df.head(10) # получение первых 10 строк таблицы df

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [4]:
df.info()  # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


В таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

В названиях колонок видны нарушения стиля:
* строчные буквы сочетаются с прописными;
* встречаются пробелы;
* слова должны быть разделены нижним подчеркиванием (в частоности, userID), что характерно для змеиного_регистра"


**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных
Исправление стиля в заголовках столбцов, исключение пропусков, проверка данных на дубликаты.

### Стиль заголовков

In [5]:
df.rename(columns={'  userID':'user_ID', 'Track':'track', '  City  ': 'city', 'Day': 'day'}, inplace=True)  # переименование столбцов
df.columns # проверка результатов - перечень названий столбцов

Index(['user_ID', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений

In [6]:
df.isna().sum() # подсчёт пропусков в данных

user_ID       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями;
* оценить, насколько они повредят расчётам. 

In [7]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown':
for col in ['track', 'artist', 'genre']:
    df[col] = df[col].fillna('unknown')

In [8]:
df.isna().sum() # подсчёт пропусков

user_ID    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты

In [9]:
df.duplicated().sum() # подсчёт явных дубликатов

3826

In [10]:
df = df.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов и сбос индексов в датафрейме

In [11]:
df.duplicated().sum() # проверка на отсутствие дубликатов

0

Теперь избавимся от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.
Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке. Для этого необходимо:
1. извлечь нужный столбец датафрейма; 
2. применить к нему метод сортировки;
3. для отсортированного столбца вызовать метод, который вернёт уникальные значения из столбца.

In [12]:
df['genre'].sort_values().unique() # просмотр уникальных названий жанров

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

В списке есть неявные дубликаты названия `hiphop`:
* *hip*,
* *hop*,
* *hip-hop*.

In [13]:
df['genre'] = df['genre'].replace(['hip', 'hop', 'hip-hop'], 'hiphop') # устранение неявных дубликатов

In [14]:
df['genre'].sort_values().unique() # проверка на неявные дубликаты

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Выполнено исправиление заголовков, чтобы упростить работу с таблицей.

Пропущенные значения заменены на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

А без дубликатов исследование станет более точным.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга.
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

Оценим активность пользователей в каждом городе.

In [15]:
df.groupby('city')['track'].count() # подсчёт прослушиваний в каждый из трёх дней

city
Moscow              42741
Saint-Petersburg    18512
Name: track, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируем данные по дню недели и посчитаем прослушивания в понедельник, среду и пятницу. Необходимо учесть, что в данных есть информация о прослушиваниях только за эти дни.

In [16]:
df.groupby('day')['track'].count() # подсчёт прослушиваний в каждый из трёх дней недели

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Напишем функцию, которая объединит два эти расчёта.

Создадим функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

В функции сохраним в переменную строки исходной таблицы, у которых значение:
  * в колонке `day` равно параметру `day`,
  * в колонке `city` равно параметру `city`.

Для этого применим логические выражения, а затем посчитаем значения в столбце `user_id` получившейся таблицы. Результат сохраним в новую переменную. Вернем эту переменную из функции.

In [17]:
# <создание функции number_tracks()>
def number_tracks(day, city): # объявление функции с двумя параметрами: day, city
    return df[(df['day'] == day) & (df['city'] == city)]['user_ID'].count() # подсчет прослушиваний для конкретного города и дня

In [18]:
number_tracks('Monday', 'Moscow')

15740

Вызовем `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [19]:
# количество прослушиваний в Москве по понедельникам:
val = number_tracks('Monday', 'Moscow')
print('Количество прослушиваний в Москве по понедельникам:', val) 

# количество прослушиваний в Санкт-Петербурге по понедельникам:
val = number_tracks('Monday', 'Saint-Petersburg') 
print(f'Количество прослушиваний в Санкт-Петербурге по понедельникам: {val}') 

# количество прослушиваний в Москве по средам:
val = number_tracks('Wednesday', 'Moscow') 
print('Количество прослушиваний в Москве по средам: {}'.format(val)) 

# количество прослушиваний в Санкт-Петербурге по средам:
val = number_tracks('Wednesday', 'Saint-Petersburg') 
print('Количество прослушиваний в Санкт-Петербурге по средам: {0}'.format(val)) 

# количество прослушиваний в Москве по пятницам:
val = number_tracks('Friday', 'Moscow') 
print(f'Количество прослушиваний в Москве по пятницам: {val:.0f}') 

# количество прослушиваний в Санкт-Петербурге по пятницам:
val = number_tracks('Friday', 'Saint-Petersburg') 
print(f'Количество прослушиваний в Санкт-Петербурге по пятницам: {val:.0f}') 

Количество прослушиваний в Москве по понедельникам: 15740
Количество прослушиваний в Санкт-Петербурге по понедельникам: 5614
Количество прослушиваний в Москве по средам: 11056
Количество прослушиваний в Санкт-Петербурге по средам: 7003
Количество прослушиваний в Москве по пятницам: 15945
Количество прослушиваний в Санкт-Петербурге по пятницам: 5895


Создадим c помощью конструктора `pd.DataFrame` таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые вы получили с помощью `number_tracks`.

In [20]:
(pd.DataFrame
 (
     data=
     {
         'city': ['Moscow', 'Saint-Petersburg'], 
         'monday': [15740, 5614], 'wednesday': [11056, 7003], 
         'friday': [15945, 5895]
     }
 )
)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы: активность пользователей зависит от дня недели, причем в Москве и Петербурге это проявляется по-разному.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [21]:
moscow_general = df[df['city'] == 'Moscow'] # получение таблицы с данными по Москве из исходного датафрейма
spb_general = df[df['city'] == 'Saint-Petersburg'] # получение таблицы с данными по Санкт-Петербургу из исходного датафрейма

In [22]:
moscow_general

,user_ID,track,artist,genre,city,time,day
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,unknown,ruspop,Moscow,09:17:40,Friday
...,...,...,...,...,...,...,...
61247,83A474E7,I Worship Only What You Bleed,The Black Dahlia Murder,extrememetal,Moscow,21:07:12,Monday
61248,729CBB09,My Name,McLean,rnb,Moscow,13:32:28,Wednesday
61250,C5E3A0D5,Jalopiina,unknown,industrial,Moscow,20:09:26,Friday
61251,321D0506,Freight Train,Chas McDevitt,rock,Moscow,21:43:59,Friday


Создим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [23]:
# Объявление функции genre_weekday() с параметрами table, day, time1, time2,
# которая возвращает информацию о самых популярных жанрах в указанный день в заданное время:
# 1) выбираются те строки переданного датафрейма table, для которых одновременно:
#    - значение в столбце day равно значению аргумента day
#    - значение в столбце time больше значения аргумента time1
#    - значение в столбце time меньше значения аргумента time2
# 2) группируется датафрейм по столбцу genre
# 3) по столбцу датафрейма проводится подсчет треков в текущем жанре методом count()
# 4) производится сортировка по убыванию 
# 5) возвращается топ-10 популярных жанров (в указанный день, в заданное время)

def genre_weekday(table, day, time1, time2):
    return (
        table[(table['day'] == day) & 
              (table['time'] > time1) & 
              (table['time'] < time2)]
        .groupby('genre')['track']
        .count()
        .sort_values(ascending=False)
        .head(10)
    )

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [24]:
# вызов функции для утра понедельника в Москве
val = genre_weekday(moscow_general, 'Monday', '07:00', '11:00')
print('Топ-10 треков на утро понедельника в Москве:', val, sep='\n\n')
print()
# вызов функции для утра понедельника в Петербурге
val = genre_weekday(spb_general, 'Monday', '07:00', '11:00')
print('Топ-10 треков на утро понедельника в Петербурге:', val, sep='\n\n')
print()
# вызов функции для вечера пятницы в Москве
val = genre_weekday(moscow_general, 'Friday', '17:00', '23:00')
print('Топ-10 треков на вечер пятницы в Москве:', val, sep='\n\n')
print()
# вызов функции для вечера пятницы в Петербурге
val = genre_weekday(spb_general, 'Friday', '17:00', '23:00')
print('Топ-10 треков на вечер пятницы в Петербурге:', val, sep='\n\n')

Топ-10 треков на утро понедельника в Москве:

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: track, dtype: int64

Топ-10 треков на утро понедельника в Петербурге:

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: track, dtype: int64

Топ-10 треков на вечер пятницы в Москве:

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: track, dtype: int64

Топ-10 треков на вечер пятницы в Петербурге:

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classica

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра. Затем отсортируем результат в порядке убывания и сохраним его в таблице `moscow_genres`.

In [25]:
moscow_genres = moscow_general.groupby('genre')['track'].count().sort_values(ascending=False)

Выведем на экран первые десять строк `moscow_genres`:

In [26]:
moscow_genres.head(10) # просмотр первых 10 строк moscow_genres

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: track, dtype: int64

Теперь повторим то же и для Петербурга.

In [27]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)
spb_genres.head(10) # просмотр первых 10 строк spb_genres

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 

## Итоги исследования

Выполнена проверка трех гипотез и установлено:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.